# 🧪 Mock LeetCode Assistant Server (Testing Version)

A **hardcoded testing server** that simulates LeetCode problem analysis without connecting to OpenAI's API. Returns a pre-written Two Sum solution for development and testing purposes.

## 📋 Endpoint Details

**`POST /process-multiple-frames-stream`**

**Input:** 
- Form data with image frames (`frame_0`, `frame_1`, etc.)
- Frame count metadata

**Output:** 
- Server-Sent Events stream with mock Two Sum solution
- Frames saved to local `frames/` directory
- Mock detected text and problem explanation

## 🚀 Usage

Run the server and send POST request with image frames to get streaming mock AI response.

In [1]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from PIL import Image
import json
import asyncio
import io
import os
from datetime import datetime
import time

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process-multiple-frames-stream")
async def simple_frame_reader_with_save(request: Request):
    """Simple endpoint that reads frames AND saves them to frames folder, then streams text"""
    
    try:
        print("=" * 50)
        print("🔄 READING AND SAVING FRAMES...")
        
        # Get content type
        content_type = request.headers.get("content-type", "")
        print(f"📋 Content-Type: {content_type}")
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        print(f"🔍 Form keys: {list(form_data.keys())}")
        
        # Create frames directory
        os.makedirs("frames", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Process and save each frame
        saved_frames = []
        frame_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open image
                    img = Image.open(io.BytesIO(file_data))
                    
                    # Save frame with timestamp
                    save_path = f"frames/frame_{timestamp}_{key}.png"
                    img.save(save_path)
                    
                    saved_frames.append({
                        "key": key,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{img.width}x{img.height}",
                        "saved_path": save_path
                    })
                    
                    frame_count += 1
                    print(f"✅ Saved {key}: {save_path} ({img.width}x{img.height})")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
            
            elif key == 'frame_count':
                expected_count = str(value)
                print(f"📊 Expected frame count: {expected_count}")
        
        print(f"🎉 Successfully saved {frame_count} frames to frames/ folder!")
        
        # Now create streaming response with initial success data + streaming text
        async def generate_stream():
            # First yield the success response
            initial_response = {
                "success": True,
                "message": f"Successfully received and saved {frame_count} frames!",
                "frames_saved": saved_frames,
                "timestamp": timestamp,
                "save_directory": "frames/",
                "streaming": True,
                "frame_count": frame_count,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            # Small delay before starting analysis
            await asyncio.sleep(0.3)
            
            # Then stream mock GPT-like Two Sum solution
            mock_gpt_responses = [
                "🤖 I can see you're working on the Two Sum problem! Let me help you solve this step by step.\n",
                "📋 **Problem Analysis:**\nGiven an array of integers `nums` and an integer `target`, return indices of two numbers that add up to `target`.\n",
                "💡 **Approach 1: Brute Force**\nWe could use nested loops to check every pair, but that would be O(n²) time complexity.\n",
                "⚡ **Better Approach: Hash Map**\nLet's use a hash map to solve this in O(n) time!\n",
                "```python\ndef twoSum(nums, target):\n    num_map = {}\n    for i, num in enumerate(nums):\n        complement = target - num\n        if complement in num_map:\n            return [num_map[complement], i]\n        num_map[num] = i\n```\n",
                "🔍 **How it works:**\n1. Create an empty hash map\n2. For each number, calculate its complement (target - current number)\n3. Check if complement exists in hash map\n4. If yes, return the indices; if no, store current number and index\n",
                "📊 **Time Complexity:** O(n) - single pass through array\n**Space Complexity:** O(n) - hash map storage\n",
                "🧪 **Test with your example:**\n`nums = [2,7,11,15], target = 9`\n- i=0, num=2, complement=7, not in map, store {2:0}\n- i=1, num=7, complement=2, found at index 0, return [0,1]\n",
                "✨ **Alternative One-liner (Python):**\n```python\ndef twoSum(nums, target):\n    seen = {}\n    return next(([seen[target-n], i] for i, n in enumerate(nums) if target-n in seen or seen.setdefault(n, i)), None)\n```\n",
                "🎯 **Key Insights:**\n- Hash maps provide O(1) average lookup time\n- We only need one pass through the array\n- Always check if complement exists before storing current number\n",
                "🏆 **Solution Complete!** This approach efficiently solves Two Sum in linear time.\n"
            ]
            
            for i, response in enumerate(mock_gpt_responses):
                await asyncio.sleep(0.8)  # Slightly longer delay for reading
                stream_data = {
                    "type": "stream",
                    "content": response,
                    "step": i + 1,
                    "total_steps": len(mock_gpt_responses)
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
            
            # Final completion message with detected text
            final_data = {
                "type": "complete",
                "content": "🎯 Two Sum solution explained successfully!\n",
                "total_frames_processed": frame_count,
                "detected_text": f"""**Detected from {frame_count} frames:**

**LeetCode Problem 1: Two Sum**

**Problem Statement:**
Given an array of integers nums and an integer target, return indices of the two numbers such that they add up to target.

You may assume that each input would have exactly one solution, and you may not use the same element twice.

**Example 1:**
Input: nums = [2,7,11,15], target = 9
Output: [0,1]
Explanation: Because nums[0] + nums[1] == 9, we return [0, 1].

**Example 2:**
Input: nums = [3,2,4], target = 6
Output: [1,2]

**Constraints:**
- 2 ≤ nums.length ≤ 10⁴
- -10⁹ ≤ nums[i] ≤ 10⁹
- -10⁹ ≤ target ≤ 10⁹
- Only one valid answer exists.

**Follow-up:** Can you come up with an algorithm that is less than O(n²) time complexity?

**Code Editor shows:**
```python
class Solution:
    def twoSum(self, nums: List[int], target: int) -> List[int]:
        # Your solution here
        pass
```"""
            }
            yield f"data: {json.dumps(final_data)}\n\n"
            
            # Send the [DONE] signal that frontend is waiting for
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Processing failed: {str(e)}"
        })

# Start server
async def start_frame_saver_server():
    import uvicorn
    try:
        print("🚀 Starting FRAME READER & SAVER server on http://localhost:8000")
        print("📁 Frames will be saved to: frames/ directory")
        print("📡 Now includes streaming Two Sum solution!")
        
        print("💡 Send your frontend request to save frames and get streaming output!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_frame_saver_server()

INFO:     Started server process [21908]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🚀 Starting FRAME READER & SAVER server on http://localhost:8000
📁 Frames will be saved to: frames/ directory
📡 Now includes streaming Two Sum solution!
💡 Send your frontend request to save frames and get streaming output!
🔄 READING AND SAVING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundary4kuMiTruikFaqJ7b
🔄 READING AND SAVING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryUSNpc3M7s1v046aq
📥 Form data items: 5
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_count']
📥 Form data items: 5
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_count']
📝 Processing frame_0: 1180743 bytes
✅ Saved frame_0: frames/frame_20250709_122518_frame_0.png (1280x720)
📝 Processing frame_0: 1180743 bytes
✅ Saved frame_0: frames/frame_20250709_122518_frame_0.png (1280x720)
📝 Processing frame_1: 1166641 bytes
✅ Saved frame_1: frames/frame_20250709_122518_frame_1.png (1280x720)
📝 Processing frame_2: 1182030 bytes
✅ Saved

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [21908]


# 🤖 AI-Powered LeetCode Assistant Server (Production Version)

A **production-ready server** that connects to **OpenAI's GPT-4.1-mini** to analyze LeetCode problem screenshots and provide intelligent solutions in real-time.

## 🔑 Prerequisites

- OpenAI API key in `.env` file as `OPENAI_API_KEY`
- Internet connection for API calls

## 📋 Endpoint Details

**`POST /process-multiple-frames-stream`**

**Input:** 
- Form data with image frames (`frame_0`, `frame_1`, etc.)
- Frame count metadata

**Output:** 
- Server-Sent Events stream with real AI analysis
- Frames saved to local `frames/` directory  
- Live streaming of problem solutions and code explanations

## 🚀 Usage

Set your OpenAI API key, run the server, and send POST request with LeetCode screenshot frames to get real AI-powered analysis.

In [6]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
from PIL import Image
import json
import asyncio
import io
import os
import base64
from datetime import datetime
import time
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.post("/process-multiple-frames-stream")
async def simple_frame_reader_with_save(request: Request):
    """Endpoint that reads frames, saves them, converts to base64, and sends to OpenAI API with streaming response"""
    
    try:
        print("=" * 50)
        print("🔄 READING AND PROCESSING FRAMES...")
        
        # Get content type
        content_type = request.headers.get("content-type", "")
        print(f"📋 Content-Type: {content_type}")
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        print(f"🔍 Form keys: {list(form_data.keys())}")
        
        # Create frames directory
        os.makedirs("frames", exist_ok=True)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        # Process frames and convert to base64
        saved_frames = []
        base64_images = []
        frame_count = 0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open image
                    img = Image.open(io.BytesIO(file_data))
                    
                    # Save frame with timestamp
                    save_path = f"frames/frame_{timestamp}_{key}.png"
                    img.save(save_path)
                    
                    # Convert to base64 for OpenAI API
                    base64_img = base64.b64encode(file_data).decode('utf-8')
                    base64_images.append(base64_img)
                    
                    saved_frames.append({
                        "key": key,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{img.width}x{img.height}",
                        "saved_path": save_path
                    })
                    
                    frame_count += 1
                    print(f"✅ Saved {key}: {save_path} ({img.width}x{img.height})")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
            
            elif key == 'frame_count':
                expected_count = str(value)
                print(f"📊 Expected frame count: {expected_count}")
        
        print(f"🎉 Successfully saved {frame_count} frames to frames/ folder!")
        print(f"🔄 Converting {len(base64_images)} images to base64 for OpenAI API...")
        
        # Now create streaming response with initial success data + real OpenAI streaming
        async def generate_stream():
            # First yield the success response
            initial_response = {
                "success": True,
                "message": f"Successfully received and saved {frame_count} frames!",
                "frames_saved": saved_frames,
                "timestamp": timestamp,
                "save_directory": "frames/",
                "streaming": True,
                "frame_count": frame_count,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            # Small delay before starting analysis
            await asyncio.sleep(0.3)
            
            try:
                # Prepare content for OpenAI API
                # Prepare content for OpenAI API
                content = [
                    {"type": "text", 
                    "text": "Analyze these LeetCode screenshots captured while scrolling. "
                    "Different frames may show different parts of the same problem (description, examples, constraints, code editor). "
                    "Combine information from all frames to provide: 1) Problem name/number 2) Complete working code solution 3) Brief explanation. "
                    "Be concise - code first, minimal explanation."
                    "Ensure not to use any imports or libraries in the code solution"}
                ]
                # Add all base64 images to the content
                for base64_img in base64_images:
                    content.append({
                        "type": "image_url", 
                        "image_url": {"url": f"data:image/png;base64,{base64_img}"}
                    })
                
                print(f"🤖 Sending {len(base64_images)} images to OpenAI API...")
                
                # Stream response from OpenAI
                stream = client.chat.completions.create(
                    model="gpt-4.1-mini",
                    messages=[
                        {
                            "role": "user",
                            "content": content
                        }
                    ],
                    temperature=0.2,
                    stream=True
                )
                
                accumulated_content = ""
                step = 0
                
                for chunk in stream:
                    # Handle content chunks
                    if chunk.choices and chunk.choices[0].delta.content is not None:
                        content_chunk = chunk.choices[0].delta.content
                        accumulated_content += content_chunk
                        step += 1
                        
                        stream_data = {
                            "type": "stream",
                            "content": content_chunk,
                            "step": step,
                            "accumulated": accumulated_content
                        }
                        yield f"data: {json.dumps(stream_data)}\n\n"
                        
                        # Small delay to make streaming visible
                        await asyncio.sleep(0.01)
                
                print("✅ OpenAI streaming completed!")
                
            except Exception as api_error:
                print(f"❌ OpenAI API Error: {api_error}")
                error_data = {
                    "type": "stream",
                    "content": f"❌ Error calling OpenAI API: {str(api_error)}\n\nUsing fallback response...\n",
                    "error": True
                }
                yield f"data: {json.dumps(error_data)}\n\n"
                
                # Fallback message
                fallback_data = {
                    "type": "stream",
                    "content": "🤖 Unable to analyze images with AI. Please check your OpenAI API key and try again.\n"
                }
                yield f"data: {json.dumps(fallback_data)}\n\n"
            
            # Final completion message
            final_data = {
                "type": "complete",
                "content": "🎯 Analysis complete!\n",
                "total_frames_processed": frame_count,
                "detected_text": f"""**Placeholder for detected text from {frame_count} frames:**

This will be replaced with actual OCR text extraction in future versions.
For now, the AI analysis above contains the problem understanding and solution.

**Technical Details:**
- Frames processed: {frame_count}
- Images sent to AI: {len(base64_images)}
- Timestamp: {timestamp}
- Save location: frames/

**Next Steps:**
- Implement OCR text extraction
- Add text preprocessing
- Enhance problem detection accuracy"""
            }
            yield f"data: {json.dumps(final_data)}\n\n"
            
            # Send the [DONE] signal that frontend is waiting for
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Processing failed: {str(e)}"
        })

# Start server
async def start_frame_saver_server():
    import uvicorn
    try:
        print("🚀 Starting FRAME READER & OPENAI ANALYZER server on http://localhost:8000")
        print("📁 Frames will be saved to: frames/ directory")
        print("🤖 Now includes real OpenAI GPT-4.1-mini analysis!")
        print("🔑 Make sure your OPENAI_API_KEY is set in .env file")
        
        print("💡 Send your frontend request to analyze LeetCode screenshots!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_frame_saver_server()

🚀 Starting FRAME READER & OPENAI ANALYZER server on http://localhost:8000
📁 Frames will be saved to: frames/ directory
🤖 Now includes real OpenAI GPT-4.1-mini analysis!
🔑 Make sure your OPENAI_API_KEY is set in .env file
💡 Send your frontend request to analyze LeetCode screenshots!


INFO:     Started server process [18776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 READING AND PROCESSING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundaryreJ2yR7s3fBdP0SW
🔄 READING AND PROCESSING FRAMES...
📋 Content-Type: multipart/form-data; boundary=----WebKitFormBoundarynx3HJyiKJR1BWIVm
📥 Form data items: 14
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_4', 'frame_5', 'frame_6', 'frame_7', 'frame_8', 'frame_9', 'frame_10', 'frame_11', 'frame_12', 'frame_count']
📝 Processing frame_0: 1015134 bytes
✅ Saved frame_0: frames/frame_20250717_201429_frame_0.png (1280x720)
📝 Processing frame_1: 994031 bytes
✅ Saved frame_1: frames/frame_20250717_201429_frame_1.png (1280x720)
📝 Processing frame_2: 984846 bytes
✅ Saved frame_2: frames/frame_20250717_201429_frame_2.png (1280x720)
📝 Processing frame_3: 988654 bytes
✅ Saved frame_3: frames/frame_20250717_201429_frame_3.png (1280x720)
📥 Form data items: 14
🔍 Form keys: ['frame_0', 'frame_1', 'frame_2', 'frame_3', 'frame_4', 'frame_5', 'frame_6', 'frame_7', 'frame_8', 'frame_9', 

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18776]


FOR DEVELOPMENT
- With Blur check
- and simple refinements, increase contrast

In [14]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
import cv2
import numpy as np
from PIL import ImageEnhance, ImageFilter,Image

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Enhanced image processing with blur detection
class ImageRefiner:
    @staticmethod
    def placeholder_refinement(img):
        """Placeholder function - currently returns same image"""
        # For now, just return the same image
        # Later you can add actual refinement logic here
        return img.copy()
    
    @staticmethod
    def leetcode_optimized_refinement(img):
        """Improved refinement that reduces glare instead of amplifying it"""
        try:
            from PIL import ImageEnhance, ImageFilter
            import cv2
            import numpy as np
            
            img_cv = np.array(img)
            
            # 1. GLARE DETECTION AND REDUCTION FIRST
            if len(img_cv.shape) == 3:
                # Convert to LAB for better glare handling
                lab = cv2.cvtColor(img_cv, cv2.COLOR_RGB2LAB)
                l, a, b = cv2.split(lab)
                
                # Detect overexposed/glare areas (very bright pixels)
                glare_mask = cv2.threshold(l, 200, 255, cv2.THRESH_BINARY)[1]
                
                # Reduce glare by applying bilateral filter to bright areas
                l_filtered = cv2.bilateralFilter(l, 9, 80, 80)
                
                # Blend original and filtered based on glare mask
                glare_mask_norm = glare_mask.astype(np.float32) / 255.0
                l_reduced = l * (1 - glare_mask_norm) + l_filtered * glare_mask_norm
                
                # Merge back to RGB
                lab_reduced = cv2.merge([l_reduced.astype(np.uint8), a, b])
                img_cv = cv2.cvtColor(lab_reduced, cv2.COLOR_LAB2RGB)
            
            # Convert back to PIL for remaining operations
            img_glare_reduced = Image.fromarray(img_cv)
            
            # 2. GENTLE CONTRAST ENHANCEMENT (much lower)
            contrast_enhancer = ImageEnhance.Contrast(img_glare_reduced)
            enhanced = contrast_enhancer.enhance(1.1)  # Reduced from 1.4
            
            # 3. CONDITIONAL BRIGHTNESS - only if image is actually dark
            img_array = np.array(enhanced)
            mean_brightness = np.mean(img_array)
            
            if mean_brightness < 120:  # Only brighten if actually dark
                brightness_enhancer = ImageEnhance.Brightness(enhanced)
                brightened = brightness_enhancer.enhance(1.1)  # Reduced from 1.2
            else:
                brightened = enhanced  # Skip brightening if already bright enough
            
            # 4. GENTLE SHARPENING only for text areas
            sharpened = brightened.filter(ImageFilter.UnsharpMask(radius=1, percent=120, threshold=3))
            
            return sharpened
            
        except Exception as e:
            print(f"⚠️ Refinement failed: {e}")
            return img.copy()
    
    @staticmethod
    def calculate_blur_score(img):
        """Calculate blur score using Laplacian variance method"""
        try:
            # Convert PIL Image to OpenCV format
            img_array = np.array(img)
            
            # Convert to grayscale if needed
            if len(img_array.shape) == 3:
                gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                gray = img_array
            
            # Calculate Laplacian variance (higher = sharper)
            laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            return laplacian_var
            
        except Exception as e:
            print(f"❌ Error calculating blur score: {e}")
            return 0.0
    
    @staticmethod
    def is_too_blurry(img, threshold=100.0):
        """Check if image is too blurry based on threshold"""
        blur_score = ImageRefiner.calculate_blur_score(img)
        return blur_score < threshold, blur_score

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint that loads images directly into list for clustering with blur detection"""
    
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR DETECTION...")
        
        # Create directory structure: date first, then categories
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_refined")
        after_dir = os.path.join(session_dir, "after_refined")
        rejected_dir = os.path.join(session_dir, "rejected_blurry")
        
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        
        # Lists to store all images directly from form data
        all_images = []
        processing_results = []
        rejected_images = []
        frame_count = 0
        blur_threshold = 250.0  # Adjust this threshold as needed
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open original image directly from file data
                    original_img = Image.open(io.BytesIO(file_data))
                    
                    # Check if image is too blurry
                    is_blurry, blur_score = ImageRefiner.is_too_blurry(original_img, blur_threshold)
                    
                    if is_blurry:
                        # Save to rejected folder
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_blur_{blur_score:.2f}.png")
                        original_img.save(rejected_path)
                        
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "blur_score": blur_score,
                            "threshold": blur_threshold,
                            "rejected_path": rejected_path,
                            "reason": "too_blurry"
                        })
                        
                        print(f"❌ REJECTED {key} - too blurry (score: {blur_score:.2f} < {blur_threshold})")
                        continue
                    
                    # Image passed blur test - process normally
                    print(f"✅ PASSED blur test {key} - score: {blur_score:.2f}")
                    
                    # Apply LeetCode optimized refinement
                    refined_img = ImageRefiner.leetcode_optimized_refinement(original_img)
                    
                    # Add directly to images list for clustering
                    image_data = {
                        "key": key,
                        "original_image": original_img,
                        "refined_image": refined_img,
                        "file_data": file_data,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{original_img.width}x{original_img.height}",
                        "blur_score": blur_score
                    }
                    all_images.append(image_data)
                    
                    # Save both images (only non-blurry ones)
                    before_path = os.path.join(before_dir, f"{key}_original_blur_{blur_score:.2f}.png")
                    after_path = os.path.join(after_dir, f"{key}_refined_blur_{blur_score:.2f}.png")
                    
                    original_img.save(before_path)
                    refined_img.save(after_path)
                    
                    processing_results.append({
                        "frame_key": key,
                        "original_path": before_path,
                        "refined_path": after_path,
                        "original_size": f"{original_img.width}x{original_img.height}",
                        "original_file_size": len(file_data),
                        "refined_file_size": os.path.getsize(after_path),
                        "blur_score": blur_score,
                        "status": "accepted"
                    })
                    
                    frame_count += 1
                    print(f"✅ Processed {key} - added to images list and saved (blur: {blur_score:.2f})")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {len(rejected_images)} blurry frames")
        print(f"📋 All {len(all_images)} sharp images loaded directly into list for clustering")
        
        # Create streaming response
        async def generate_development_stream():
            # Initial response
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} sharp images, {len(rejected_images)} rejected (too blurry)",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_refined": before_dir,
                    "after_refined": after_dir,
                    "rejected_blurry": rejected_dir
                },
                "frame_count": frame_count,
                "rejected_count": len(rejected_images),
                "all_images_count": len(all_images),
                "blur_threshold": blur_threshold,
                "type": "initial"
            }
            yield f"data: {json.dumps(initial_response)}\n\n"
            
            await asyncio.sleep(0.3)
            
            # Stream processing results (accepted images)
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": result
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Stream rejected images info
            if rejected_images:
                await asyncio.sleep(0.2)
                for i, rejected in enumerate(rejected_images):
                    stream_data = {
                        "type": "rejected_frame",
                        "frame_index": i + 1,
                        "total_rejected": len(rejected_images),
                        "rejected": rejected
                    }
                    yield f"data: {json.dumps(stream_data)}\n\n"
                    await asyncio.sleep(0.1)
            
            # Summary with blur detection and clustering info
            total_original_size = sum([img["size_bytes"] for img in all_images])
            avg_blur_score = sum([img["blur_score"] for img in all_images]) / len(all_images) if all_images else 0
            
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Development Processing Summary with Blur Detection**

📊 **Frame Statistics:**
- Total frames received: {frame_count + len(rejected_images)}
- Sharp frames accepted: {frame_count}
- Blurry frames rejected: {len(rejected_images)}
- Acceptance rate: {(frame_count/(frame_count + len(rejected_images))*100) if (frame_count + len(rejected_images)) > 0 else 0:.1f}%

📁 **Total Size (accepted):** {total_original_size:,} bytes
📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - Before refinement: {before_dir}
   - After refinement: {after_dir}
   - Rejected blurry: {rejected_dir}

🔍 **Blur Detection Results:**
- Threshold used: {blur_threshold}
- Average blur score (accepted): {avg_blur_score:.2f}
- Higher scores = sharper images

🖼️ **Images Loaded Directly into List:**
- Total sharp images: {len(all_images)}
- Each image contains: PIL Image objects, metadata, blur scores
- Ready for immediate clustering analysis

🔧 **Enhanced Processing Flow:**
1. Read file data from form ✅
2. Create PIL Image directly from bytes ✅
3. Calculate blur score using Laplacian variance ✅
4. Filter out blurry images (< {blur_threshold}) ✅
5. Apply LeetCode-optimized refinement ✅
6. Add to all_images list ✅
7. Save to organized directory structure ✅

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_refined/
    ├── after_refined/
    └── rejected_blurry/

🔍 **Clustering Ready:**
- All {len(all_images)} SHARP images available in `all_images` list
- No file I/O needed for clustering algorithms
- Direct access to PIL Image objects for feature extraction
- Quality assured - no blurry images will affect clustering

💡 **Next Steps:**
- Implement clustering algorithm on `all_images` list
- Feature extraction for similarity analysis
- Group similar SHARP frames before GPT processing

🚀 **Efficient quality-controlled pipeline** - Only sharp images in memory for clustering!""",
                "total_original_size": total_original_size,
                "images_in_memory": len(all_images),
                "processing_results": processing_results,
                "rejected_images": rejected_images,
                "blur_stats": {
                    "threshold": blur_threshold,
                    "avg_blur_score": avg_blur_score,
                    "acceptance_rate": (frame_count/(frame_count + len(rejected_images))*100) if (frame_count + len(rejected_images)) > 0 else 0
                }
            }
            yield f"data: {json.dumps(summary_data)}\n\n"
            
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

# Start server
async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting DEVELOPMENT FRAME PROCESSOR server on http://localhost:8000")
        print("📁 Images organized by session timestamp")
        print("🔧 LeetCode-optimized refinement active")
        print("🔍 NOW WITH BLUR DETECTION - filters out blurry images!")
        print("📂 Directory structure: development/TIMESTAMP/[before_refined|after_refined|rejected_blurry]/")
        print("📋 Perfect for clustering algorithms with quality control")
        print("💰 NO GPT API calls - save costs during development!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting DEVELOPMENT FRAME PROCESSOR server on http://localhost:8000
📁 Images organized by session timestamp
🔧 LeetCode-optimized refinement active
🔍 NOW WITH BLUR DETECTION - filters out blurry images!
📂 Directory structure: development/TIMESTAMP/[before_refined|after_refined|rejected_blurry]/
📋 Perfect for clustering algorithms with quality control
💰 NO GPT API calls - save costs during development!


INFO:     Started server process [18776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR DETECTION...
🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR DETECTION...
📥 Form data items: 14
📝 Processing frame_0: 933115 bytes
❌ REJECTED frame_0 - too blurry (score: 158.42 < 250.0)
📝 Processing frame_1: 910528 bytes
❌ REJECTED frame_1 - too blurry (score: 204.13 < 250.0)
📝 Processing frame_2: 960704 bytes
❌ REJECTED frame_2 - too blurry (score: 216.54 < 250.0)
📝 Processing frame_3: 849002 bytes
❌ REJECTED frame_3 - too blurry (score: 99.64 < 250.0)
📝 Processing frame_4: 883669 bytes
❌ REJECTED frame_4 - too blurry (score: 67.05 < 250.0)
📝 Processing frame_5: 863794 bytes
❌ REJECTED frame_5 - too blurry (score: 94.15 < 250.0)
📝 Processing frame_6: 897859 bytes
❌ REJECTED frame_6 - too blurry (score: 140.47 < 250.0)
📝 Processing frame_7: 952541 bytes
❌ REJECTED frame_7 - too blurry (score: 119.98 < 250.0)
📝 Processing frame_8: 825402 bytes
❌ REJECTED frame_8 - too blurry (score: 71.67 < 250.0)
📝 Processing frame_9: 837299 bytes
❌ REJECTED frame_9 -

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18776]


DEVELOPMENT, exploration on angles

In [17]:
import nest_asyncio
nest_asyncio.apply()

from fastapi import FastAPI, Request
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import JSONResponse, StreamingResponse
import json
import asyncio
import io
import os
from datetime import datetime
import time
import cv2
import numpy as np
from PIL import ImageEnhance, ImageFilter, Image
import math

app = FastAPI()

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Enhanced image processing with blur detection and angle correction
class ImageRefiner:
    @staticmethod
    def placeholder_refinement(img):
        """Placeholder function - currently returns same image"""
        return img.copy()
    
    @staticmethod
    def detect_text_angle(img):
        """Detect text skew angle using Hough Line Transform"""
        try:
            img_array = np.array(img)
            
            # Convert to grayscale
            if len(img_array.shape) == 3:
                gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                gray = img_array
            
            # Apply edge detection
            edges = cv2.Canny(gray, 50, 150, apertureSize=3)
            
            # Apply Hough Line Transform
            lines = cv2.HoughLines(edges, 1, np.pi/180, threshold=100)
            
            if lines is None:
                return 0.0, "no_lines_detected"
            
            # Calculate angles of detected lines
            angles = []
            for rho, theta in lines[:, 0]:
                angle = theta * 180 / np.pi
                # Convert to text orientation angle (-90 to 90)
                if angle > 90:
                    angle = angle - 180
                elif angle < -90:
                    angle = angle + 180
                angles.append(angle)
            
            # Filter angles close to horizontal (text lines)
            horizontal_angles = [a for a in angles if abs(a) < 45]
            
            if not horizontal_angles:
                return 0.0, "no_horizontal_lines"
            
            # Calculate median angle to avoid outliers
            median_angle = float(np.median(horizontal_angles))  # Convert to Python float
            
            return median_angle, "success"
            
        except Exception as e:
            print(f"❌ Error detecting angle: {e}")
            return 0.0, f"error: {str(e)}"
    
    @staticmethod
    def correct_perspective(img, angle, max_correction=15.0):
        """Correct image perspective based on detected angle"""
        try:
            # Only correct if angle is significant enough
            if abs(angle) < 0.5:
                return img, f"angle_too_small_{angle:.2f}"
            
            # Limit correction to prevent over-rotation
            if abs(angle) > max_correction:
                angle = max_correction if angle > 0 else -max_correction
                correction_status = f"limited_to_{angle:.2f}"
            else:
                correction_status = f"corrected_{angle:.2f}"
            
            img_array = np.array(img)
            height, width = img_array.shape[:2]
            
            # Calculate rotation matrix
            center = (width // 2, height // 2)
            rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
            
            # Calculate new bounding box to avoid cropping
            cos = abs(rotation_matrix[0, 0])
            sin = abs(rotation_matrix[0, 1])
            new_width = int((height * sin) + (width * cos))
            new_height = int((height * cos) + (width * sin))
            
            # Adjust translation
            rotation_matrix[0, 2] += (new_width / 2) - center[0]
            rotation_matrix[1, 2] += (new_height / 2) - center[1]
            
            # Apply rotation
            rotated = cv2.warpAffine(img_array, rotation_matrix, (new_width, new_height), 
                                   flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_CONSTANT, 
                                   borderValue=(255, 255, 255))
            
            return Image.fromarray(rotated), correction_status
            
        except Exception as e:
            print(f"❌ Error correcting perspective: {e}")
            return img, f"error: {str(e)}"
    
    @staticmethod
    def detect_document_corners(img):
        """Detect document corners for perspective correction (advanced method)"""
        try:
            img_array = np.array(img)
            
            # Convert to grayscale
            if len(img_array.shape) == 3:
                gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                gray = img_array
            
            # Apply Gaussian blur
            blurred = cv2.GaussianBlur(gray, (5, 5), 0)
            
            # Apply adaptive threshold
            thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)
            
            # Find contours
            contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            
            # Find largest contour (presumably the document)
            if not contours:
                return None, "no_contours"
            
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Approximate contour to quadrilateral
            epsilon = 0.02 * cv2.arcLength(largest_contour, True)
            approx = cv2.approxPolyDP(largest_contour, epsilon, True)
            
            if len(approx) == 4:
                return approx.reshape(4, 2), "found_corners"
            else:
                return None, f"found_{len(approx)}_corners"
                
        except Exception as e:
            print(f"❌ Error detecting corners: {e}")
            return None, f"error: {str(e)}"
    
    @staticmethod
    def apply_perspective_transform(img, corners):
        """Apply four-point perspective transform"""
        try:
            img_array = np.array(img)
            height, width = img_array.shape[:2]
            
            # Order corners: top-left, top-right, bottom-right, bottom-left
            def order_points(pts):
                rect = np.zeros((4, 2), dtype="float32")
                s = pts.sum(axis=1)
                rect[0] = pts[np.argmin(s)]  # top-left
                rect[2] = pts[np.argmax(s)]  # bottom-right
                diff = np.diff(pts, axis=1)
                rect[1] = pts[np.argmin(diff)]  # top-right
                rect[3] = pts[np.argmax(diff)]  # bottom-left
                return rect
            
            rect = order_points(corners)
            
            # Calculate width and height of new image
            (tl, tr, br, bl) = rect
            widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
            widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
            maxWidth = max(int(widthA), int(widthB))
            
            heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
            heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
            maxHeight = max(int(heightA), int(heightB))
            
            # Destination points
            dst = np.array([
                [0, 0],
                [maxWidth - 1, 0],
                [maxWidth - 1, maxHeight - 1],
                [0, maxHeight - 1]], dtype="float32")
            
            # Calculate perspective transform matrix
            M = cv2.getPerspectiveTransform(rect, dst)
            
            # Apply perspective transform
            warped = cv2.warpPerspective(img_array, M, (maxWidth, maxHeight))
            
            return Image.fromarray(warped), "perspective_corrected"
            
        except Exception as e:
            print(f"❌ Error applying perspective transform: {e}")
            return img, f"error: {str(e)}"
    
    @staticmethod
    def leetcode_optimized_refinement(img):
        """Enhanced refinement with angle detection and perspective correction"""
        try:
            refinement_log = []
            
            # 1. ANGLE DETECTION AND CORRECTION
            angle, angle_status = ImageRefiner.detect_text_angle(img)
            refinement_log.append(f"angle_detection: {angle:.2f}° ({angle_status})")
            
            # Apply angle correction if needed
            if abs(angle) > 0.5 and angle_status == "success":
                img, correction_status = ImageRefiner.correct_perspective(img, -angle)  # Negative to correct
                refinement_log.append(f"angle_correction: {correction_status}")
            else:
                refinement_log.append("angle_correction: skipped")
            
            # 2. DOCUMENT CORNER DETECTION (optional advanced correction)
            corners, corner_status = ImageRefiner.detect_document_corners(img)
            refinement_log.append(f"corner_detection: {corner_status}")
            
            # Apply perspective correction if corners detected
            if corners is not None and corner_status == "found_corners":
                img, perspective_status = ImageRefiner.apply_perspective_transform(img, corners)
                refinement_log.append(f"perspective_correction: {perspective_status}")
            else:
                refinement_log.append("perspective_correction: skipped")
            
            # 3. CONVERT TO OPENCV FOR PROCESSING
            img_cv = np.array(img)
            
            # 4. GLARE DETECTION AND REDUCTION
            if len(img_cv.shape) == 3:
                # Convert to LAB for better glare handling
                lab = cv2.cvtColor(img_cv, cv2.COLOR_RGB2LAB)
                l, a, b = cv2.split(lab)
                
                # Detect overexposed/glare areas (very bright pixels)
                glare_mask = cv2.threshold(l, 200, 255, cv2.THRESH_BINARY)[1]
                
                # Reduce glare by applying bilateral filter to bright areas
                l_filtered = cv2.bilateralFilter(l, 9, 80, 80)
                
                # Blend original and filtered based on glare mask
                glare_mask_norm = glare_mask.astype(np.float32) / 255.0
                l_reduced = l * (1 - glare_mask_norm) + l_filtered * glare_mask_norm
                
                # Merge back to RGB
                lab_reduced = cv2.merge([l_reduced.astype(np.uint8), a, b])
                img_cv = cv2.cvtColor(lab_reduced, cv2.COLOR_LAB2RGB)
                refinement_log.append("glare_reduction: applied")
            
            # Convert back to PIL for remaining operations
            img_glare_reduced = Image.fromarray(img_cv)
            
            # 5. GENTLE CONTRAST ENHANCEMENT
            contrast_enhancer = ImageEnhance.Contrast(img_glare_reduced)
            enhanced = contrast_enhancer.enhance(1.1)
            refinement_log.append("contrast_enhancement: 1.1x")
            
            # 6. CONDITIONAL BRIGHTNESS - only if image is actually dark
            img_array = np.array(enhanced)
            mean_brightness = np.mean(img_array)
            
            if mean_brightness < 120:
                brightness_enhancer = ImageEnhance.Brightness(enhanced)
                brightened = brightness_enhancer.enhance(1.1)
                refinement_log.append(f"brightness_enhancement: 1.1x (mean: {mean_brightness:.1f})")
            else:
                brightened = enhanced
                refinement_log.append(f"brightness_enhancement: skipped (mean: {mean_brightness:.1f})")
            
            # 7. GENTLE SHARPENING for text clarity
            sharpened = brightened.filter(ImageFilter.UnsharpMask(radius=1, percent=120, threshold=3))
            refinement_log.append("sharpening: applied")
            
            # Store refinement log in image metadata (for debugging)
            if hasattr(sharpened, 'info'):
                sharpened.info['refinement_log'] = ' | '.join(refinement_log)
            
            print(f"🔧 Refinement log: {' | '.join(refinement_log)}")
            
            return sharpened
            
        except Exception as e:
            print(f"⚠️ Refinement failed: {e}")
            return img.copy()
    
    @staticmethod
    def calculate_blur_score(img):
        """Calculate blur score using Laplacian variance method"""
        try:
            # Convert PIL Image to OpenCV format
            img_array = np.array(img)
            
            # Convert to grayscale if needed
            if len(img_array.shape) == 3:
                gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
            else:
                gray = img_array
            
            # Calculate Laplacian variance (higher = sharper)
            laplacian_var = cv2.Laplacian(gray, cv2.CV_64F).var()
            return float(laplacian_var)  # Convert to Python float
            
        except Exception as e:
            print(f"❌ Error calculating blur score: {e}")
            return 0.0
    
    @staticmethod
    def is_too_blurry(img, threshold=100.0):
        """Check if image is too blurry based on threshold"""
        blur_score = ImageRefiner.calculate_blur_score(img)
        return blur_score < threshold, blur_score
    
    @staticmethod
    def calculate_angle_score(img):
        """Calculate how tilted the image is"""
        try:
            angle, status = ImageRefiner.detect_text_angle(img)
            return float(abs(angle)), status  # Convert to Python float
        except Exception as e:
            return 0.0, f"error: {str(e)}"

# Helper function to convert numpy types to Python types for JSON serialization
def convert_to_json_serializable(obj):
    """Convert numpy types to Python types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_json_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_json_serializable(item) for item in obj]
    else:
        return obj

@app.post("/process-multiple-frames-stream")
async def process_frames_development(request: Request):
    """Development endpoint with blur detection, angle detection, and perspective correction"""
    
    try:
        print("=" * 50)
        print("🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR & ANGLE DETECTION...")
        
        # Create directory structure
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_dir = "development"
        session_dir = os.path.join(base_dir, timestamp)
        before_dir = os.path.join(session_dir, "before_refined")
        after_dir = os.path.join(session_dir, "after_refined")
        rejected_dir = os.path.join(session_dir, "rejected_blurry")
        angle_dir = os.path.join(session_dir, "angle_analysis")
        
        os.makedirs(before_dir, exist_ok=True)
        os.makedirs(after_dir, exist_ok=True)
        os.makedirs(rejected_dir, exist_ok=True)
        os.makedirs(angle_dir, exist_ok=True)
        
        # Parse form data
        form_data = await request.form()
        print(f"📥 Form data items: {len(form_data)}")
        
        # Lists to store all images
        all_images = []
        processing_results = []
        rejected_images = []
        frame_count = 0
        blur_threshold = 300.0
        
        for key, value in form_data.items():
            if key.startswith('frame_') and hasattr(value, 'read'):
                try:
                    # Read file data
                    file_data = await value.read()
                    print(f"📝 Processing {key}: {len(file_data)} bytes")
                    
                    # Open original image
                    original_img = Image.open(io.BytesIO(file_data))
                    
                    # Check blur score
                    is_blurry, blur_score = ImageRefiner.is_too_blurry(original_img, blur_threshold)
                    
                    # Check angle
                    angle_score, angle_status = ImageRefiner.calculate_angle_score(original_img)
                    
                    if is_blurry:
                        # Save to rejected folder
                        rejected_path = os.path.join(rejected_dir, f"{key}_rejected_blur_{blur_score:.2f}_angle_{angle_score:.2f}.png")
                        original_img.save(rejected_path)
                        
                        rejected_images.append({
                            "key": key,
                            "filename": getattr(value, 'filename', 'unknown'),
                            "blur_score": float(blur_score),  # Ensure Python float
                            "angle_score": float(angle_score),  # Ensure Python float
                            "angle_status": angle_status,
                            "threshold": float(blur_threshold),  # Ensure Python float
                            "rejected_path": rejected_path,
                            "reason": "too_blurry"
                        })
                        
                        print(f"❌ REJECTED {key} - blur: {blur_score:.2f}, angle: {angle_score:.2f}°")
                        continue
                    
                    # Image passed tests - process normally
                    print(f"✅ PASSED tests {key} - blur: {blur_score:.2f}, angle: {angle_score:.2f}° ({angle_status})")
                    
                    # Apply enhanced refinement with angle correction
                    refined_img = ImageRefiner.leetcode_optimized_refinement(original_img)
                    
                    # Calculate post-refinement angle for comparison
                    post_angle, post_angle_status = ImageRefiner.calculate_angle_score(refined_img)
                    
                    # Add to images list
                    image_data = {
                        "key": key,
                        "original_image": original_img,
                        "refined_image": refined_img,
                        "file_data": file_data,
                        "filename": getattr(value, 'filename', 'unknown'),
                        "size_bytes": len(file_data),
                        "image_size": f"{original_img.width}x{original_img.height}",
                        "blur_score": float(blur_score),  # Ensure Python float
                        "original_angle": float(angle_score),  # Ensure Python float
                        "refined_angle": float(post_angle),  # Ensure Python float
                        "angle_status": angle_status,
                        "angle_improvement": float(angle_score - post_angle)  # Ensure Python float
                    }
                    all_images.append(image_data)
                    
                    # Save images with detailed naming
                    before_path = os.path.join(before_dir, f"{key}_original_blur_{blur_score:.2f}_angle_{angle_score:.2f}.png")
                    after_path = os.path.join(after_dir, f"{key}_refined_blur_{blur_score:.2f}_angle_{post_angle:.2f}.png")
                    
                    original_img.save(before_path)
                    refined_img.save(after_path)
                    
                    # Create angle analysis visualization
                    angle_analysis_path = os.path.join(angle_dir, f"{key}_angle_analysis.png")
                    # Save side-by-side comparison
                    combined_width = original_img.width + refined_img.width
                    combined_height = max(original_img.height, refined_img.height)
                    combined_img = Image.new('RGB', (combined_width, combined_height), (255, 255, 255))
                    combined_img.paste(original_img, (0, 0))
                    combined_img.paste(refined_img, (original_img.width, 0))
                    combined_img.save(angle_analysis_path)
                    
                    processing_results.append({
                        "frame_key": key,
                        "original_path": before_path,
                        "refined_path": after_path,
                        "angle_analysis_path": angle_analysis_path,
                        "original_size": f"{original_img.width}x{original_img.height}",
                        "original_file_size": len(file_data),
                        "refined_file_size": os.path.getsize(after_path),
                        "blur_score": float(blur_score),  # Ensure Python float
                        "original_angle": float(angle_score),  # Ensure Python float
                        "refined_angle": float(post_angle),  # Ensure Python float
                        "angle_improvement": float(angle_score - post_angle),  # Ensure Python float
                        "angle_status": angle_status,
                        "status": "accepted"
                    })
                    
                    frame_count += 1
                    print(f"✅ Processed {key} - angle improved by {angle_score - post_angle:.2f}°")
                    
                except Exception as frame_error:
                    print(f"❌ Error processing {key}: {frame_error}")
        
        print(f"🎉 Successfully processed {frame_count} frames!")
        print(f"❌ Rejected {len(rejected_images)} blurry frames")
        print(f"📋 All {len(all_images)} sharp images with angle correction ready")
        
        # Create streaming response
        async def generate_development_stream():
            # Initial response
            initial_response = {
                "success": True,
                "message": f"Development processing complete! {frame_count} sharp images, {len(rejected_images)} rejected",
                "timestamp": timestamp,
                "session_directory": session_dir,
                "directories": {
                    "session": session_dir,
                    "before_refined": before_dir,
                    "after_refined": after_dir,
                    "rejected_blurry": rejected_dir,
                    "angle_analysis": angle_dir
                },
                "frame_count": frame_count,
                "rejected_count": len(rejected_images),
                "all_images_count": len(all_images),
                "blur_threshold": float(blur_threshold),  # Ensure Python float
                "type": "initial"
            }
            yield f"data: {json.dumps(convert_to_json_serializable(initial_response))}\n\n"
            
            await asyncio.sleep(0.3)
            
            # Stream processing results
            for i, result in enumerate(processing_results):
                stream_data = {
                    "type": "frame_result",
                    "frame_index": i + 1,
                    "total_frames": frame_count,
                    "result": convert_to_json_serializable(result)
                }
                yield f"data: {json.dumps(stream_data)}\n\n"
                await asyncio.sleep(0.1)
            
            # Stream rejected images info
            if rejected_images:
                await asyncio.sleep(0.2)
                for i, rejected in enumerate(rejected_images):
                    stream_data = {
                        "type": "rejected_frame",
                        "frame_index": i + 1,
                        "total_rejected": len(rejected_images),
                        "rejected": convert_to_json_serializable(rejected)
                    }
                    yield f"data: {json.dumps(stream_data)}\n\n"
                    await asyncio.sleep(0.1)
            
            # Calculate statistics
            total_original_size = sum([img["size_bytes"] for img in all_images])
            avg_blur_score = sum([img["blur_score"] for img in all_images]) / len(all_images) if all_images else 0
            avg_angle_improvement = sum([img["angle_improvement"] for img in all_images]) / len(all_images) if all_images else 0
            
            summary_data = {
                "type": "summary",
                "content": f"""🔬 **Development Processing Summary with Blur & Angle Detection**

📊 **Frame Statistics:**
- Total frames received: {frame_count + len(rejected_images)}
- Sharp frames accepted: {frame_count}
- Blurry frames rejected: {len(rejected_images)}
- Acceptance rate: {(frame_count/(frame_count + len(rejected_images))*100) if (frame_count + len(rejected_images)) > 0 else 0:.1f}%

📁 **Total Size (accepted):** {total_original_size:,} bytes
📍 **Session Directory Structure:**
   - Main session: {session_dir}
   - Before refinement: {before_dir}
   - After refinement: {after_dir}
   - Rejected blurry: {rejected_dir}
   - Angle analysis: {angle_dir}

🔍 **Quality Control Results:**
- Blur threshold: {blur_threshold}
- Average blur score: {avg_blur_score:.2f}
- Average angle improvement: {avg_angle_improvement:.2f}°

🖼️ **Enhanced Processing Features:**
- ✅ Blur detection and filtering
- ✅ Text angle detection using Hough transforms
- ✅ Automatic perspective correction
- ✅ Document corner detection (advanced)
- ✅ Glare reduction in LAB color space
- ✅ Conditional brightness adjustment
- ✅ Gentle contrast enhancement
- ✅ Text-optimized sharpening

🔧 **Angle Correction Pipeline:**
1. Detect text lines using Canny edge detection
2. Apply Hough Line Transform to find line angles
3. Calculate median angle for robustness
4. Correct perspective if angle > 0.5°
5. Optional: Four-point perspective correction
6. Apply image enhancement pipeline

📂 **Directory Organization:**
development/
└── {timestamp}/
    ├── before_refined/ (original images)
    ├── after_refined/ (enhanced images)
    ├── rejected_blurry/ (filtered out)
    └── angle_analysis/ (side-by-side comparisons)

🔍 **Clustering Ready:**
- All {len(all_images)} SHARP, STRAIGHT images available
- Enhanced image quality for better OCR
- Perspective-corrected for optimal text recognition
- Quality metadata for clustering algorithms

💡 **Next Steps:**
- Implement clustering on corrected images
- Add OCR text extraction pipeline
- Feature extraction for similarity analysis

🚀 **Quality-controlled pipeline with geometric correction!**""",
                "total_original_size": total_original_size,
                "images_in_memory": len(all_images),
                "processing_results": convert_to_json_serializable(processing_results),
                "rejected_images": convert_to_json_serializable(rejected_images),
                "quality_stats": {
                    "blur_threshold": float(blur_threshold),
                    "avg_blur_score": float(avg_blur_score),
                    "avg_angle_improvement": float(avg_angle_improvement),
                    "acceptance_rate": float((frame_count/(frame_count + len(rejected_images))*100) if (frame_count + len(rejected_images)) > 0 else 0)
                }
            }
            yield f"data: {json.dumps(convert_to_json_serializable(summary_data))}\n\n"
            
            yield "data: [DONE]\n\n"
        
        return StreamingResponse(
            generate_development_stream(),
            media_type="text/event-stream",
            headers={
                "Cache-Control": "no-cache",
                "Connection": "keep-alive",
                "Content-Type": "text/event-stream",
                "Access-Control-Allow-Origin": "*",
                "Access-Control-Allow-Methods": "*",
                "Access-Control-Allow-Headers": "*"
            }
        )
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return JSONResponse({
            "success": False,
            "error": f"Development processing failed: {str(e)}"
        })

# Start server
async def start_development_server():
    import uvicorn
    try:
        print("🚀 Starting ENHANCED DEVELOPMENT FRAME PROCESSOR server on http://localhost:8000")
        print("📁 Images organized by session timestamp")
        print("🔧 LeetCode-optimized refinement with angle correction")
        print("🔍 Blur detection + perspective correction active!")
        print("📐 Angle detection using Hough Line Transform")
        print("📂 Directory: development/TIMESTAMP/[before|after|rejected|angle_analysis]/")
        print("📋 Perfect for OCR and clustering with geometric correction!")
        print("💰 NO GPT API calls - cost-effective development!")
        
        config = uvicorn.Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = uvicorn.Server(config)
        await server.serve()
    except Exception as e:
        print(f"❌ Server error: {e}")

await start_development_server()

🚀 Starting ENHANCED DEVELOPMENT FRAME PROCESSOR server on http://localhost:8000
📁 Images organized by session timestamp
🔧 LeetCode-optimized refinement with angle correction
🔍 Blur detection + perspective correction active!
📐 Angle detection using Hough Line Transform
📂 Directory: development/TIMESTAMP/[before|after|rejected|angle_analysis]/
📋 Perfect for OCR and clustering with geometric correction!
💰 NO GPT API calls - cost-effective development!


INFO:     Started server process [18776]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR & ANGLE DETECTION...
🔄 DEVELOPMENT FRAME PROCESSING WITH BLUR & ANGLE DETECTION...
📥 Form data items: 14
📥 Form data items: 14
📝 Processing frame_0: 1058973 bytes
✅ PASSED tests frame_0 - blur: 600.77, angle: 2.00° (success)
🔧 Refinement log: angle_detection: 2.00° (success) | angle_correction: corrected_-2.00 | corner_detection: found_6_corners | perspective_correction: skipped | glare_reduction: applied | contrast_enhancement: 1.1x | brightness_enhancement: 1.1x (mean: 110.9) | sharpening: applied
✅ Processed frame_0 - angle improved by -0.50°
📝 Processing frame_1: 990503 bytes
✅ PASSED tests frame_1 - blur: 553.84, angle: 1.00° (success)
🔧 Refinement log: angle_detection: -1.00° (success) | angle_correction: corrected_1.00 | corner_detection: found_10_corners | perspective_correction: skipped | glare_reduction: applied | contrast_enhancement: 1.1x | brightness_enhancement: 1.1x (mean: 119.0) | sharpening: applied
✅ Processed frame_1 - angle i

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18776]
